In [2]:
from time import time
from os import path
from gensim import corpora, models
from gensim.models.phrases import Phrases
from corputil import FileCorpus, MultiFileCorpus
from corputil.modifiers import to_words_sl, sentence_to_words
from pprint import pprint
from collections import defaultdict

In [ ]:
label = 'KW44'
num_topics = 50
file1 = path.join('data', 'Corpus_{}_{}.txt'.format(label, 'P'))
file2 = path.join('data', 'Corpus_{}_{}.txt'.format(label, 'R'))

train_corpus_p = FileCorpus(file1, modifier=to_words_sl)
train_corpus_r = FileCorpus(file2, modifier=to_words_sl)
train_corpus_b = MultiFileCorpus([file1, file2], modifier=to_words_sl)

def train_lda(train):
    corpus = [bigram[doc] for doc in train]

    dictionary = corpora.Dictionary(corpus)
    dictionary.filter_extremes()
    dictionary.compactify()

    mmCorpus = [dictionary.doc2bow(doc) for doc in corpus]
    lda = models.LdaMulticore(mmCorpus, id2word=dictionary, chunksize=500, batch=True, 
                              num_topics=num_topics, workers=2, passes=10, iterations=500)
    print(lda.print_topics(50))

In [ ]:
bigram = Phrases.load(path.join('models', 'Bigram.phrase'))
train_lda(train_corpus_p)
print('####################################')
train_lda(train_corpus_r)
print('####################################')
train_lda(train_corpus_b)

In [ ]:
# clean_topics = []
# for topic_id, topics in lda.print_topics(40):
#     topic_list = []
#     for topic in topics.split('+'):
#         topic_list.append(tuple(topic.strip().split('*')))
#     clean_topics.append(topic_list)

# flat = [l for inner in clean_topics for l in inner]
# trends = defaultdict(float)
# for p, trend in flat:
#     trends[trend] += float(p)
# top_trends = sorted(trends.items(), key=lambda item: -item[1])
# pprint(top_trends)